In [13]:
import os
from dotenv import load_dotenv

load_dotenv()
AZURE_OPENAI_ENDPOINT=os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_VERSION=os.getenv("AZURE_OPENAI_API_VERSION")
AZURE_OPENAI_DEPLOYMENT=os.getenv("AZURE_OPENAI_DEPLOYMENT")

In [14]:
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider

# Use the current user identity to authenticate with Azure OpenAI, Cognitive Search and Blob Storage (no secrets needed, 
# just use 'az login' locally, and managed identity when deployed on Azure). If you need to use keys, use separate AzureKeyCredential instances with the 
# keys for each service
# If you encounter an error here are some ways to troubleshoot:
# - you can exclude the problematic credential by using a parameter (ex. exclude_shared_token_cache_credential=True)
# - For your credential, grant Cognitive Services User and Cognitive Services OpenAI User roles - EVEN IF YOU HAVE OWNER/ADMIN RIGHTS.
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")

client = AzureOpenAI(
  azure_endpoint = AZURE_OPENAI_ENDPOINT,
  api_version=AZURE_OPENAI_API_VERSION,
  azure_ad_token_provider=token_provider
)

In [15]:
IMAGE_PATH="../media/notes-sample.png"

In [16]:
import base64

def executeGptVision(encoded_image, system_prompt_file=None, system_prompt=None):
    if system_prompt_file:
        with open(system_prompt_file, "r") as file:
            system_prompt = file.read()
    elif system_prompt is None:
        raise ValueError("Either filename or system_prompt must be provided.")

    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/jpeg;base64,{encoded_image}"
                    }
                }
            ]
        }
    ]

    response = client.chat.completions.create(
        model=AZURE_OPENAI_DEPLOYMENT,
        messages=messages,
        temperature=0
    )

    return response.choices[0].message.content

In [17]:
encoded_image = base64.b64encode(open(IMAGE_PATH, 'rb').read()).decode('ascii')
noteType = executeGptVision(encoded_image, system_prompt_file="../az-function/prompts/detectNoteType.txt")
print (noteType)

PAPER


In [18]:
if noteType == "PAPER":
    ocrFile = "../az-function/prompts/ocrPaper.txt"
elif noteType == "WHITEBOARD":
    ocrFile = "../az-function/prompts/ocrWhiteboard.txt"
else:
    ocrFile = "../az-function/prompts/ocrImage.txt"

extractedText = executeGptVision(encoded_image, system_prompt_file=ocrFile)
print (extractedText)

```markdown
## 20240701 My Test Notes

This is me, randomly writing something to test my app.

### Attendees:
Riff, Raynor, Jim, John, Kerrigan, Annie

## MOTM

Top priorities
- More apps using Gen AI
- Use GitHub Copilot!
- How to integrate with Azure
- CI/CD?

### Note:
- This is written in markdown
- I should include drawings

| July         | August         | Sept          |
|--------------|----------------|---------------|
| - Comms      | - Enablement   | - Execution   |
| - Meetings   | - Workshops    | - Metrics     |

```mermaid
flowchart LR
    Notes --> Code
    Code --> Dev Test
    Dev Test --> Reusable Template
    Code --> Documentation
```

^ Hope this makes sense?

_Let's write in script._
```


In [19]:
def executeCompletion(text, system_prompt_file=None, system_prompt=None):
    if system_prompt_file:
        with open(system_prompt_file, "r") as file:
            system_prompt = file.read()
    elif system_prompt is None:
        raise ValueError("Either filename or system_prompt must be provided.")

    messages = [
        {
            "role": "system",
            "content": system_prompt
        },
        {
            "role": "user",
            "content": text
        }
    ]

    response = client.chat.completions.create(
        model="gpt4o",
        messages=messages,
        temperature=0.3
    )

    return response.choices[0].message.content

In [20]:
if noteType == "PAPER" or noteType == "WHITEBOARD":
    extractedText = executeCompletion(text=extractedText, system_prompt_file="../az-function/prompts/proofread.txt")
    print (extractedText)

```markdown
## 20240701 My Test Notes

This is me, randomly writing something to test my app.

### Attendees:
Riff, Raynor, Jim, John, Kerrigan, Annie

## MOTM

Top priorities
- More apps using Generative AI
- Use GitHub Copilot!
- How to integrate with Azure
- Continuous Integration/Continuous Deployment?

### Note:
- This is written in markdown
- I should include drawings

| July         | August         | September     |
|--------------|----------------|---------------|
| - Comms      | - Enablement   | - Execution   |
| - Meetings   | - Workshops    | - Metrics     |

```mermaid
flowchart LR
    Notes --> Code
    Code --> DevTest[Dev Test]
    DevTest --> ReusableTemplate[Reusable Template]
    Code --> Documentation
```

^ Hope this makes sense?

_Let's write in script._
```


In [21]:
if noteType == "PAPER" or noteType == "WHITEBOARD":
    extractedText = executeCompletion(extractedText, system_prompt_file="../az-function/prompts/sectionHeader.txt")
    print (extractedText)

```markdown
## 20240701 My Test Notes

This is me, randomly writing something to test my app.

### Attendees:
Riff, Raynor, Jim, John, Kerrigan, Annie

## MOTM

Top priorities
- More apps using Generative AI
- Use GitHub Copilot!
- How to integrate with Azure
- Continuous Integration/Continuous Deployment?

### Note:
- This is written in markdown
- I should include drawings

### Schedule
| July         | August         | September     |
|--------------|----------------|---------------|
| - Comms      | - Enablement   | - Execution   |
| - Meetings   | - Workshops    | - Metrics     |

```mermaid
flowchart LR
    Notes --> Code
    Code --> DevTest[Dev Test]
    DevTest --> ReusableTemplate[Reusable Template]
    Code --> Documentation
```

^ Hope this makes sense?

_Let's write in script._
```


In [31]:
## Test, shows text completion is much faster.
import time

determine_title_prompt = """
Extract the main title for this image.
- Respond with the title only.
- If the title is in markdown, remove markdown formatting.
- If there is no title, respond with "NONE".
"""

start_time = time.time()
title = executeGptVision(encoded_image, system_prompt=determine_title_prompt)

end_time = time.time()
duration = end_time - start_time
print(f"{title} (Execution time: {duration} seconds)")

determine_title_prompt = f"""Extract the main title for these notes
- Respond with the title only.
- If the title is in markdown, remove markdown formatting.
- If there is no title, respond with "NONE".
"""

start_time = time.time()
title = executeCompletion(extractedText, system_prompt=determine_title_prompt)
end_time = time.time()
duration = end_time - start_time
print(f"{title} (Execution time: {duration} seconds)")

20240701 My Test Notes (Execution time: 3.886441230773926 seconds)
20240701 My Test Notes (Execution time: 0.4628307819366455 seconds)


In [40]:
import datetime

def get_title(extracted_text):
    title = executeCompletion(extracted_text, system_prompt_file="../az-function/prompts/extractMainTitle.txt")
    title = title.replace("{DateStamp}", datetime.datetime.now().strftime("%Y%m%d"))
    return title

In [45]:
print(get_title("2024-07-13 Weekly Planning Notes"))
print(get_title("2024-07-13 09:00 Daily Journal"))
print(get_title("202407 Team Offsite"))
print(get_title("## MOTM - Customer Meeting"))
print(get_title("# **Key Note** - Microsoft Build _(2024)_"))

print(get_title(extractedText))

20240713 Weekly Planning Notes
20240713 0900 Daily Journal
20240701 Team Offsite
20240707 MOTM - Customer Meeting
20240707 Key Note - Microsoft Build 2024
20240701 My Test Notes


In [23]:
extractedText = extractedText.strip()
if extractedText.startswith("```markdown"):
    extractedText = extractedText[len("```markdown"):]
if extractedText.endswith("```"):
    extractedText = extractedText[:-len("```")]

extractedText = extractedText.strip()

In [24]:
import os
import datetime

filename = os.path.basename(IMAGE_PATH)
last_updated = datetime.datetime.fromtimestamp(os.path.getmtime(IMAGE_PATH)).strftime('%Y-%m-%d %H:%M')

result = f"""---
note-type: {noteType}
created-date: {last_updated}
last-updated: {last_updated}
---
# {filename}
![{filename}]({IMAGE_PATH})
{extractedText}"""

# save result to output.md
with open("output.md", "w") as file:
    file.write(result)

print(result)


---
note-type: PAPER
created-date: 2024-07-01 17:23
last-updated: 2024-07-01 17:23
---
# notes-sample.png
![notes-sample.png](../media/notes-sample.png)
## 20240701 My Test Notes

This is me, randomly writing something to test my app.

### Attendees:
Riff, Raynor, Jim, John, Kerrigan, Annie

## MOTM

Top priorities
- More apps using Generative AI
- Use GitHub Copilot!
- How to integrate with Azure
- Continuous Integration/Continuous Deployment?

### Note:
- This is written in markdown
- I should include drawings

### Schedule
| July         | August         | September     |
|--------------|----------------|---------------|
| - Comms      | - Enablement   | - Execution   |
| - Meetings   | - Workshops    | - Metrics     |

```mermaid
flowchart LR
    Notes --> Code
    Code --> DevTest[Dev Test]
    DevTest --> ReusableTemplate[Reusable Template]
    Code --> Documentation
```

^ Hope this makes sense?

_Let's write in script._
